# CaSR SWE File Combination Workflow

This notebook demonstrates how to combine NetCDF files from the CaSR SWE dataset using the `combine_casr_swe_files.py` script. The CaSR dataset contains files organized by variable types, spatial regions, and time periods that can be combined in different ways:

1. **Temporal combination**: Combine files across time periods
2. **Spatial combination**: Combine files across spatial regions  
3. **Full combination**: Combine both temporal and spatial dimensions

The CaSR SWE dataset includes:
- **Variable types**: A_PR24_SFC (precipitation) and P_SWE_LAND (snow water equivalent)
- **Spatial regions**: Different rlon/rlat coordinate ranges
- **Time periods**: 4-year chunks from 1980-2023

## Setup and Imports

**Note**: If you encounter NumPy compatibility errors, please run one of the following commands in your terminal before running this notebook:

**Option 1 (Recommended)**: Install from requirements file
```bash
pip install -r requirements_notebook.txt
```

**Option 2**: Manual installation with compatible versions
```bash
pip install "numpy<2" xarray pandas matplotlib netcdf4
```

**Option 3**: Using conda
```bash
conda install numpy=1.26 xarray pandas matplotlib netcdf4
```

**Option 4**: Create a new environment with compatible versions
```bash
conda create -n snowdrought python=3.9 numpy=1.26 xarray pandas matplotlib netcdf4 jupyter
conda activate snowdrought
```

In [ ]:
# Check for NumPy compatibility issues
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='numpy')

# Import required packages
import sys
import os
from pathlib import Path

# Handle NumPy compatibility
try:
    import numpy as np
    print(f"NumPy version: {np.__version__}")
except ImportError as e:
    print(f"NumPy import error: {e}")
    print("Please install NumPy: pip install numpy")

# Import data science packages with error handling
try:
    import xarray as xr
    print(f"xarray version: {xr.__version__}")
except ImportError as e:
    print(f"xarray import error: {e}")
    print("If you encounter NumPy compatibility issues, try:")
    print("  pip install 'numpy<2' xarray pandas matplotlib")
    print("  or")
    print("  conda install numpy=1.26 xarray pandas matplotlib")
    raise

try:
    import pandas as pd
    print(f"pandas version: {pd.__version__}")
except ImportError as e:
    print(f"pandas import error: {e}")
    raise

try:
    import matplotlib.pyplot as plt
    print(f"matplotlib version: {plt.matplotlib.__version__}")
except ImportError as e:
    print(f"matplotlib import error: {e}")
    raise

# Add the project root to Python path to import the combine script
project_root = Path().cwd().parent.parent
sys.path.append(str(project_root))

# Import the CaSR file combiner
try:
    from combine_casr_swe_files import CaSRFileCombiner
    print("Successfully imported CaSRFileCombiner")
except ImportError as e:
    print(f"Error importing CaSRFileCombiner: {e}")
    print("Make sure combine_casr_swe_files.py is in the project root directory")
    raise

## Configuration

Set up the input and output directories for your CaSR SWE data files.

In [ ]:
# Define data paths - modify these paths according to your data location
input_dir = r"data/input_data/CaSR_SWE"  # Directory containing CaSR NetCDF files
output_dir = r"data/output_data/combined_casr"  # Directory for combined output files

# Create absolute paths
input_path = project_root / input_dir
output_path = project_root / output_dir

print(f"Input directory: {input_path}")
print(f"Output directory: {output_path}")
print(f"Input directory exists: {input_path.exists()}")

## Initialize the CaSR File Combiner

Create an instance of the `CaSRFileCombiner` class with your input and output directories.

In [ ]:
# Initialize the file combiner
combiner = CaSRFileCombiner(input_dir=str(input_path), output_dir=str(output_path))

## Explore Dataset Information

Before combining files, let's examine what data is available in the input directory.

In [ ]:
# Get information about the available datasets
combiner.get_dataset_info()

## Examine File Groups

Let's look at how the files are grouped by variable type.

In [ ]:
# Get file groups
file_groups = combiner.get_file_groups()

print("Available file groups:")
for group_name, files in file_groups.items():
    print(f"\n{group_name}: {len(files)} files")
    
    # Show first few filenames as examples
    for i, file_path in enumerate(files[:3]):
        filename = Path(file_path).name
        print(f"  {i+1}. {filename}")
    
    if len(files) > 3:
        print(f"  ... and {len(files) - 3} more files")

## Example: Parse Individual Filenames

Let's examine how the filename parsing works for understanding the file structure.

In [ ]:
# Get a sample file and parse its filename
if file_groups:
    # Get the first file from the first group
    first_group = list(file_groups.keys())[0]
    sample_file = file_groups[first_group][0]
    sample_filename = Path(sample_file).name
    
    print(f"Sample filename: {sample_filename}")
    
    # Parse the filename
    parsed_info = combiner.parse_filename(sample_filename)
    
    print("\nParsed information:")
    for key, value in parsed_info.items():
        print(f"  {key}: {value}")
else:
    print("No files found in the input directory. Please check your input path.")

## Combination Options

Now let's demonstrate the different ways to combine the CaSR files.

### Option 1: Temporal Combination Only

Combine files across time periods while keeping spatial regions separate.

In [ ]:
# Temporal combination only
print("Performing temporal combination (keeping spatial regions separate)...")
combiner.combine_by_variable(combine_spatial=False, combine_temporal=True)
print("Temporal combination completed!")

### Option 2: Spatial Combination Only

Combine files across spatial regions while keeping time periods separate.

In [ ]:
# Spatial combination only
print("Performing spatial combination (keeping time periods separate)...")
combiner.combine_by_variable(combine_spatial=True, combine_temporal=False)
print("Spatial combination completed!")

### Option 3: Full Combination

Combine files across both spatial and temporal dimensions to create complete datasets.

In [ ]:
# Full combination (both spatial and temporal)
print("Performing full combination (both spatial and temporal)...")
combiner.combine_by_variable(combine_spatial=True, combine_temporal=True)
print("Full combination completed!")

## Examine Combined Output Files

Let's check what files were created in the output directory.

In [ ]:
# List output files
output_files = list(output_path.glob('*.nc'))

print(f"Combined files created in {output_path}:")
print(f"Total files: {len(output_files)}\n")

for i, file_path in enumerate(output_files, 1):
    file_size = file_path.stat().st_size / (1024**2)  # Size in MB
    print(f"{i}. {file_path.name} ({file_size:.1f} MB)")

## Load and Examine a Combined Dataset

Let's load one of the combined files to examine its structure and contents.

In [ ]:
# Load a combined dataset for examination
if output_files:
    # Select the first combined file
    sample_combined_file = output_files[0]
    print(f"Loading: {sample_combined_file.name}")
    
    # Load the dataset
    ds_combined = xr.open_dataset(sample_combined_file)
    
    print("\nDataset information:")
    print(f"Dimensions: {dict(ds_combined.sizes)}")
    print(f"Variables: {list(ds_combined.data_vars.keys())}")
    print(f"Coordinates: {list(ds_combined.coords.keys())}")
    
    # Show time range if time dimension exists
    if 'time' in ds_combined.dims:
        time_start = pd.to_datetime(ds_combined.time.min().values)
        time_end = pd.to_datetime(ds_combined.time.max().values)
        print(f"Time range: {time_start.strftime('%Y-%m-%d')} to {time_end.strftime('%Y-%m-%d')}")
    
    # Show dataset attributes
    print("\nDataset attributes:")
    for attr, value in ds_combined.attrs.items():
        if isinstance(value, list) and len(value) > 5:
            print(f"  {attr}: [{len(value)} files combined]")
        else:
            print(f"  {attr}: {value}")
    
    # Close the dataset
    ds_combined.close()
else:
    print("No combined files found. Please run the combination steps first.")

## Visualize Combined Data

Let's create a simple visualization of the combined dataset.

In [ ]:
# Create a simple visualization of the combined data
if output_files:
    # Load a combined SWE dataset if available
    swe_files = [f for f in output_files if 'SWE' in f.name]
    
    if swe_files:
        ds_swe = xr.open_dataset(swe_files[0])
        
        # Get the SWE variable name
        swe_var = None
        for var in ds_swe.data_vars:
            if 'SWE' in var:
                swe_var = var
                break
        
        if swe_var and 'time' in ds_swe.dims:
            # Plot time series of mean SWE
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
            
            # Time series plot
            swe_mean = ds_swe[swe_var].mean(dim=['rlon', 'rlat'])
            swe_mean.plot(ax=ax1)
            ax1.set_title(f'Mean SWE Time Series - {swe_files[0].name}')
            ax1.set_ylabel('SWE (mm)')
            
            # Spatial plot for a sample time
            if len(ds_swe.time) > 0:
                sample_time = ds_swe.time[len(ds_swe.time)//2]  # Middle time point
                swe_spatial = ds_swe[swe_var].sel(time=sample_time)
                
                im = swe_spatial.plot(ax=ax2, cmap='Blues')
                ax2.set_title(f'SWE Spatial Distribution - {pd.to_datetime(sample_time.values).strftime("%Y-%m-%d")}')
            
            plt.tight_layout()
            
            # Save the plot
            plot_output_dir = project_root / "data" / "output_plots"
            plot_output_dir.mkdir(parents=True, exist_ok=True)
            plt.savefig(plot_output_dir / 'combined_casr_swe_overview.png', dpi=300, bbox_inches='tight')
            plt.show()
            
            ds_swe.close()
        else:
            print("No suitable SWE variable or time dimension found for visualization.")
    else:
        print("No SWE files found in the combined outputs.")
else:
    print("No combined files available for visualization.")

## Custom Combination Example

Here's an example of how to perform custom combinations using the individual methods.

In [ ]:
# Example: Custom temporal combination for a specific variable and spatial region
file_groups = combiner.get_file_groups()

if file_groups:
    # Select files for a specific variable (e.g., SWE)
    swe_group = None
    for group_name, files in file_groups.items():
        if 'SWE' in group_name:
            swe_group = group_name
            break
    
    if swe_group:
        print(f"Working with group: {swe_group}")
        
        # Group files by spatial region
        spatial_groups = {}
        for file_path in file_groups[swe_group]:
            parsed = combiner.parse_filename(Path(file_path).name)
            if parsed:
                spatial_key = f"{parsed['rlon_range']}_{parsed['rlat_range']}"
                if spatial_key not in spatial_groups:
                    spatial_groups[spatial_key] = []
                spatial_groups[spatial_key].append(file_path)
        
        print(f"\nFound {len(spatial_groups)} spatial regions:")
        for spatial_key, files in spatial_groups.items():
            print(f"  {spatial_key}: {len(files)} files")
        
        # Combine temporally for the first spatial region as an example
        if spatial_groups:
            first_spatial_key = list(spatial_groups.keys())[0]
            first_spatial_files = spatial_groups[first_spatial_key]
            
            print(f"\nCombining {len(first_spatial_files)} files for region {first_spatial_key}...")
            
            custom_output_filename = f"CaSR_v3.1_{swe_group}_{first_spatial_key}_custom_temporal.nc"
            combiner.combine_temporal(first_spatial_files, custom_output_filename)
            
            print(f"Custom combination completed: {custom_output_filename}")
    else:
        print("No SWE group found in the file groups.")
else:
    print("No file groups available.")

## Summary and Next Steps

This notebook demonstrated how to use the `CaSRFileCombiner` class to combine NetCDF files from the CaSR SWE dataset. The key capabilities include:

1. **Dataset exploration**: Understanding the structure and organization of CaSR files
2. **Temporal combination**: Combining files across time periods
3. **Spatial combination**: Combining files across spatial regions
4. **Full combination**: Creating complete datasets with both spatial and temporal coverage
5. **Custom combinations**: Flexible combination strategies for specific needs

### Key Functions Used:
- `CaSRFileCombiner()`: Main class for file combination
- `get_dataset_info()`: Explore available data
- `get_file_groups()`: Group files by variable type
- `parse_filename()`: Extract metadata from filenames
- `combine_by_variable()`: Automated combination by variable type
- `combine_temporal()`: Manual temporal combination
- `combine_spatial_regions()`: Manual spatial combination

### Next Steps:
1. Use the combined datasets in your snow drought analysis workflows
2. Apply the combined data to specific study regions or basins
3. Integrate with other notebooks in this workflow series
4. Customize combination strategies based on your research needs

The combined files are saved in the output directory and can be used directly with xarray and other analysis tools.